# Nursery dataset
Was originally created to rank and evaluate nursery school applications. 
So an application with for example, a family that is financially stable, has good housing, and has no social or health problems would be classified as priority.
And applications that may involve severe financial, social, or health issues that make it highly unlikely for the application to be accepted, would be classified as

Dit weet ik niet zeker??..

So the ranking from best to worst is:

Very Recommended > Recommended > Priority > Special Priority > Not Recommended.

In [1]:
#!pip install -r ../requirements.txt

In [2]:
# ucimlrepo is a tool that provides easy access to datasets hosted on the UCI Machine Learning Repository
from ucimlrepo import fetch_ucirepo

# Fetch dataset from UCI repository, which has ud 76
nursery = fetch_ucirepo(id=76)

# Display metadata and variable information
print(nursery.metadata) # metadata 
print(nursery.variables) # variable information 
print("\n"+ "The first 5 rows of the dataset:")
print(nursery.data.features.head())  # Display first 5 rows of features
# Show the target variable, possibilities

# Show the target variable and its unique possibilities
unique_targets = nursery.data.targets['class'].unique()  # Access the 'class' column and get unique values
print("\nPossible target classes:")
print(unique_targets)  # Display unique target classes

{'uci_id': 76, 'name': 'Nursery', 'repository_url': 'https://archive.ics.uci.edu/dataset/76/nursery', 'data_url': 'https://archive.ics.uci.edu/static/public/76/data.csv', 'abstract': ' Nursery Database was derived from a hierarchical decision model originally developed to rank applications for nursery schools.', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 12960, 'num_features': 8, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1989, 'last_updated': 'Sun Jan 14 2024', 'dataset_doi': '10.24432/C5P88W', 'creators': ['Vladislav Rajkovic'], 'intro_paper': {'ID': 372, 'type': 'NATIVE', 'title': 'An application for admission in public school systems', 'authors': 'M. Olave, V. Rajkovic, M. Bohanec', 'venue': 'Expert Systems in Public Administration', 'year': 1989, 'journal': None, 'DOI': None, 

# Imports

In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
# from model import Mamba, ModelArgs  # Import your custom Mamba implementation

# Preprocessing
The datta will be preprocessed, and converted into tensors

In [4]:
X = nursery.data.features  # These are all the feature columns in the dataset
Y = nursery.data.targets  # This is the target column in the dataset
print("\nOriginal  values (before encoding):")
print(X[1:4])  # Display a sample of the feature values
print(Y[1:4])  # Display a sample of the target values

# In case of future errors: Y = Y.values.ravel()  # Flatten Y to make it a 1D array if needed
label_encoder = LabelEncoder()  # Used to encode the categorical target variables into numerical values
X = X.apply(label_encoder.fit_transform)  # Encode the feature variables (X)
Y = label_encoder.fit_transform(Y)  # Encode the target variable (Y)

print("\nEncoded target values (after encoding):")
print(X[1:4])  # Display the encoded feature values
print(Y[1:4])  # Display the encoded target values

# Split into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert the train/test data into PyTorch tensors
# We must do this because PyTorch models only accept tensors as input
# Both the MambaClassifier and Mamba classes inherit from torch.nn.Module
# which is the base class for all neural network modules in PyTorch.
X_train_tensor = torch.tensor(X_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.long)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

# Lets see how these tensors look like
print("\nSample of training data tensor:")
print(X_train_tensor[0:2])  # Display a sample of the training data tensor
print("\nSample of training target tensor:")
print(Y_train_tensor[0:2])  # Display a sample of the training target tensor
print("\nSample of testing data tensor:")
print(X_test_tensor[0:2])  # Display a sample of the testing data tensor
print("\nSample of testing target tensor:")
print(Y_test_tensor[0:2])  # Display a sample of the testing target tensor

# Create PyTorch datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

# DataLoader to help in batch processing during model training/testing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)


Original  values (before encoding):
  parents has_nurs      form children     housing     finance         social  \
1   usual   proper  complete        1  convenient  convenient        nonprob   
2   usual   proper  complete        1  convenient  convenient        nonprob   
3   usual   proper  complete        1  convenient  convenient  slightly_prob   

        health  
1     priority  
2    not_recom  
3  recommended  
       class
1   priority
2  not_recom
3  recommend

Encoded target values (after encoding):
   parents  has_nurs  form  children  housing  finance  social  health
1        2         3     0         0        0        0       0       1
2        2         3     0         0        0        0       0       0
3        2         3     0         0        0        0       2       2
[1 0 2]

Sample of training data tensor:
tensor([[2, 0, 2, 1, 1, 0, 2, 1],
        [0, 4, 3, 3, 0, 0, 0, 0]])

Sample of training target tensor:
tensor([3, 0])

Sample of testing data tensor:
tenso

/Users/cagriatik/MAMBARIS/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Defining the model

In [6]:
from model import Mamba, ModelArgs
# I also want another mamba model which was not pretrained
d_model = 64
n_layer = 4
vocabsize = len(X.nunique())
# or might it be vocabsize = X.apply(lambda col: col.nunique()).max()
model_args = ModelArgs(d_model=d_model, n_layer=n_layer, vocab_size=vocabsize)
num_classes = len(nursery.data.targets['class'].unique())
model = Mamba(model_args, num_classes=num_classes)


# Training MAMBA on Nursery

In [8]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Set the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    # for inputs, labels in train_loader:
    #     inputs, labels = inputs.to(device), labels.to(device)

    #     # Zero the gradients
    #     optimizer.zero_grad()

    #     # Forward pass
    #     outputs = model(inputs)
    #     loss = criterion(outputs, labels)

    #     # Backward pass and optimization
    #     loss.backward()
    #     optimizer.step()

    #     running_loss += loss.item()

    # Training loop

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Assuming outputs are of shape [batch_size, num_classes]
        # If outputs have an extra dimension, you might not need to reshape them
        if outputs.dim() == 2:  # Typical case for classification
            # No need to reshape if it's already [batch_size, num_classes]
            pass  # Keep as is
        else:
            # This case might happen if outputs have unexpected dimensions
            outputs = outputs.view(-1, num_classes)  # Use this only if necessary

        # Flatten labels if they are not already
        labels = labels.view(-1)  # Flatten the labels to [batch_size * seq_length]

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/1], Loss: 0.6382


# Evaluating the model

In [ ]:
# Switch to evaluation mode
model.eval()
y_pred = []
y_true = []

# Test the model
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy on the test set: {accuracy:.4f}')